In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [2]:
%%sql
drop table if exists Items,ShopItems;
CREATE TABLE Items
(item VARCHAR(16) PRIMARY KEY);
 
CREATE TABLE ShopItems
(shop VARCHAR(16),
 item VARCHAR(16),
    PRIMARY KEY(shop, item));

INSERT INTO Items VALUES('ビール');
INSERT INTO Items VALUES('紙オムツ');
INSERT INTO Items VALUES('自転車');

INSERT INTO ShopItems VALUES('仙台',  'ビール');
INSERT INTO ShopItems VALUES('仙台',  '紙オムツ');
INSERT INTO ShopItems VALUES('仙台',  '自転車');
INSERT INTO ShopItems VALUES('仙台',  'カーテン');
INSERT INTO ShopItems VALUES('東京',  'ビール');
INSERT INTO ShopItems VALUES('東京',  '紙オムツ');
INSERT INTO ShopItems VALUES('東京',  '自転車');
INSERT INTO ShopItems VALUES('大阪',  'テレビ');
INSERT INTO ShopItems VALUES('大阪',  '紙オムツ');
INSERT INTO ShopItems VALUES('大阪',  '自転車');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ 各グループ(各店舗グループ)において、Itemsを全て持っている店舗を探す(集合 vs 集合)
  + (要求するitem) \subset (店舗にあるitem)であればよい
  + (要求するitem) \equal (店舗にあるitem)の場合はSetEqual.ipynbを参照

In [51]:
%%sql
-- havingを利用
select sh.shop
from ShopItems as sh
join Items it
on sh.item = it.item
group by sh.shop
having count(sh.item) = (
    select count(item)
    from Items
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


shop
東京
仙台


In [4]:
%%sql
-- 相関サブクエリのnot exists入れ子
select distinct shop
from ShopItems as si1
where not exists (
    select *
    from items as it
    where not exists (
        select *
        from ShopItems as si2
        where it.item = si2.item and
        si1.shop = si2.shop
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


shop
仙台
東京


In [22]:
%%sql
-- 相関サブクエリ(中身はleft join)
select distinct shop
from ShopItems si1
where not exists (
    select *
    from items as it
    left join ShopItems si2
    on it.item = si2.item and
    si1.shop = si2.shop
    where si2.item is null
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


shop
東京
仙台


In [37]:
%%sql
-- 上の解説
select *
    from items as it
    left join ShopItems si2
    on si2.shop = '大阪' and
    -- この条件がなくてもleft joinなので、(it.item, None)のレコードができる
    it.item = si2.item

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


item,shop,item_1
ビール,None,None
紙オムツ,大阪,紙オムツ
自転車,大阪,自転車


In [35]:
%%sql
select distinct shop
from ShopItems si1
-- (要求されるitem) \subset (shopのitem) であればよい
where not exists (
    -- 要求されるitem
    select item
    from items as it
    except
    -- shopのitem
    select item
    from ShopItems as si2
    where si1.shop = si2.shop
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


shop
東京
仙台
